In [7]:
import pandas as pd

import psycopg2 as pg2
con = pg2.connect(host='this_postgres', user='postgres',database='postgres')
con.autocommit = True
cur = con.cursor()

def select(sql):
    return pd.read_sql(sql,con)

Check running processes

In [8]:
sql = '''
SELECT pid , query, *
FROM pg_stat_activity pgsa
WHERE pid <> pg_backend_pid()
'''

In [9]:
df = select(sql)

In [10]:
df

,pid,query,datid,datname,pid,usesysid,usename,application_name,client_addr,client_hostname,...,xact_start,query_start,state_change,wait_event_type,wait_event,state,backend_xid,backend_xmin,query,backend_type
0,67,,NaN,None,67,NaN,None,,None,None,...,NaT,NaT,NaT,Activity,AutoVacuumMain,None,None,None,,autovacuum launcher
1,68,,NaN,None,68,10.0,postgres,,None,None,...,NaT,NaT,NaT,Activity,LogicalLauncherMain,None,None,None,,background worker
2,69,COMMIT,13044.0,postgres,69,10.0,postgres,,172.18.0.3,None,...,NaT,2019-12-29 06:05:01.567577+00:00,2019-12-29 06:05:01.568574+00:00,Client,ClientRead,idle,None,None,COMMIT,client backend
3,71,select * from sales t,13044.0,postgres,71,10.0,postgres,,172.18.0.3,None,...,2019-12-29 06:05:09.302768+00:00,2019-12-29 06:05:09.303063+00:00,2019-12-29 06:05:09.303640+00:00,Client,ClientRead,idle in transaction,None,None,select * from sales t,client backend
4,47,,NaN,None,47,NaN,None,,None,None,...,NaT,NaT,NaT,Activity,BgWriterHibernate,None,None,None,,background writer
5,46,,NaN,None,46,NaN,None,,None,None,...,NaT,NaT,NaT,Activity,CheckpointerMain,None,None,None,,checkpointer
6,48,,NaN,None,48,NaN,None,,None,None,...,NaT,NaT,NaT,Activity,WalWriterMain,None,None,None,,walwriter


Kill all

In [11]:
sql = '''
SELECT 
    pg_terminate_backend(pid) 
FROM 
    pg_stat_activity 
WHERE 
    -- don't kill my own connection!
    pid <> pg_backend_pid()

'''

In [12]:
cur.execute(sql)